# SVM implementation for Music Genre Classification using song lyrics

### Common code to import libraries and read dataset

In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from gensim.models import KeyedVectors
import ast

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score, recall_score
from sklearn.model_selection import GridSearchCV

In [9]:
df = pd.read_csv('Processed_Lyric_Dataset (1).csv')

In [10]:
df.head(10)

Unnamed: 0         Artist  \
0           0  Ivete Sangalo   
1           1  Ivete Sangalo   
2           2  Ivete Sangalo   
3           3  Ivete Sangalo   
4           4  Ivete Sangalo   
5           5  Ivete Sangalo   
6           6  Ivete Sangalo   
7           7  Ivete Sangalo   
8           8  Ivete Sangalo   
9          10  Ivete Sangalo   

                                               SName  \
0                                   Careless Whisper   
1  Could You Be Loved / Citação Musical do Rap: S...   
2                             Cruisin' (Part. Saulo)   
3                                               Easy   
4                  For Your Babies (The Voice cover)   
5                                       Human Nature   
6     Losing Control (Miss Cady feat. Ivete Sangalo)   
7                           Master Blaster (Jammin')   
8                                    More Than Words   
9              Where It Begins (feat. Nelly Furtado)   

                                               Lyric Genre  \
0  I feel so unsure\nAs I take your hand and lead...   pop   
1  Don't let them fool, ya\nOr even try to school...   pop   
2  Baby, let's cruise, away from here\nDon't be c...   pop   
3  Know it sounds funny\nBut, I just can't stand ...   pop   
4  You've got that look again\nThe one I hoped I ...   pop   
5  Looking out\nAcross the night time\nThe city w...   pop   
6  Uh, yeah.\nGo, go, go.\nUh, yeah.\nUh, Uh, Uhh...   pop   
7  Everyone's feeling pretty\nIt's hotter than Ju...   pop   
8  Saying 'I Love you'\nIs not the words I want t...   pop   
9  When you're alone and you don't know how,\nTo ...   pop   

                                      LyricProcessed  \
0  [['feel', 'unsure'], ['take', 'hand', 'lead', ...   
1  [['let', 'fool', 'ya'], ['even', 'try', 'schoo...   
2  [['baby', 'let', 'cruise', 'away'], ['confuse'...   
3  [['know', 'sound', 'funny'], ['cant', 'stand',...   
4  [['get', 'look'], ['one', 'hop', 'lad'], ['fac...   
5  [['look'], ['across', 'night', 'time'], ['city...   
6  [['uh', 'yeah..', 'go', 'go', 'go..', 'uh', 'y...   
7  [['everyones', 'feel', 'pretty'], ['hotter', '...   
8  [['say', 'love'], ['word', 'want', 'hear'], ['...   
9  [['alone', 'know'], ['fill', 'hole', 'inside',...   

                                          LyricsList  
0  [['feel', 'unsure'], ['take', 'hand', 'lead', ...  
1  [['let', 'fool', 'ya'], ['even', 'try', 'schoo...  
2  [['baby', 'let', 'cruise', 'away'], ['confuse'...  
3  [['know', 'sound', 'funny'], ['cant', 'stand',...  
4  [['get', 'look'], ['one', 'hop', 'lad'], ['fac...  
5  [['look'], ['across', 'night', 'time'], ['city...  
6  [['uh', 'yeah..', 'go', 'go', 'go..', 'uh', 'y...  
7  [['everyones', 'feel', 'pretty'], ['hotter', '...  
8  [['say', 'love'], ['word', 'want', 'hear'], ['...  
9  [['alone', 'know'], ['fill', 'hole', 'inside',...

In [11]:
# Performing label encoding on the Genre column
label_encoder = LabelEncoder()
df['Genre_Encoded'] = label_encoder.fit_transform(df['Genre'])
df.head(5)

Unnamed: 0         Artist  \
0           0  Ivete Sangalo   
1           1  Ivete Sangalo   
2           2  Ivete Sangalo   
3           3  Ivete Sangalo   
4           4  Ivete Sangalo   

                                               SName  \
0                                   Careless Whisper   
1  Could You Be Loved / Citação Musical do Rap: S...   
2                             Cruisin' (Part. Saulo)   
3                                               Easy   
4                  For Your Babies (The Voice cover)   

                                               Lyric Genre  \
0  I feel so unsure\nAs I take your hand and lead...   pop   
1  Don't let them fool, ya\nOr even try to school...   pop   
2  Baby, let's cruise, away from here\nDon't be c...   pop   
3  Know it sounds funny\nBut, I just can't stand ...   pop   
4  You've got that look again\nThe one I hoped I ...   pop   

                                      LyricProcessed  \
0  [['feel', 'unsure'], ['take', 'hand', 'lead', ...   
1  [['let', 'fool', 'ya'], ['even', 'try', 'schoo...   
2  [['baby', 'let', 'cruise', 'away'], ['confuse'...   
3  [['know', 'sound', 'funny'], ['cant', 'stand',...   
4  [['get', 'look'], ['one', 'hop', 'lad'], ['fac...   

                                          LyricsList  Genre_Encoded  
0  [['feel', 'unsure'], ['take', 'hand', 'lead', ...              9  
1  [['let', 'fool', 'ya'], ['even', 'try', 'schoo...              9  
2  [['baby', 'let', 'cruise', 'away'], ['confuse'...              9  
3  [['know', 'sound', 'funny'], ['cant', 'stand',...              9  
4  [['get', 'look'], ['one', 'hop', 'lad'], ['fac...              9

In [12]:
df['LyricsList'] = df['LyricsList'].apply(lambda x: ast.literal_eval(x))

### Using Word2Vec embeddings

In [ ]:
# load word embedding files
def read_embeddings(training_file):
    '''Loads and parses embeddings trained in earlier.
    Parameters and return values are up to you.
    '''
    # you may find generating the following two dicts useful:
    # word to embedding : {'the':[0....], ...}
    # index to embedding : {1:[0....], ...} 
    # use your tokenizer's word_index to find the index of
    # a given word
    model = KeyedVectors.load_word2vec_format(training_file, binary=False)
    
    # Create a dictionary that maps words to their embeddings
    word_to_embeddings = {word: model[word] for word in model.key_to_index}

    # Create a dictionary that maps indexes to their embeddings
    # index_to_embeddings = {index: model.wv.vectors[index] for index in range(len(model.vocab))}
    
    return word_to_embeddings

In [ ]:
word_to_embeddings = read_embeddings('w2v_embeddings (1).txt')

## SVM Model without any tuning: Vector by averaging

In [ ]:
# convert lyrics to vectors based on the word embeddings
data = [] 
labels = [] 

for _, row in df.iterrows(): 
  label = row["Genre"] 
  lyrics = row["LyricsList"] 
  feature_vector = np.zeros((200,)) 
  song_length = 0 
  for line in lyrics: 
    song_length += len(line) 
    for word in line: 
      try:
        feature_vector += word_to_embeddings[word] 
      except:
        pass
  data.append(feature_vector/song_length) 
  labels.append(label)

<ipython-input-8-22ed93fe98d8>:17: RuntimeWarning: invalid value encountered in true_divide
  data.append(feature_vector/song_length)


In [ ]:
# check for nan values in data
# if present, remove that row in data and labels
index_to_remove = list()
c = 0
for i in range(len(data)):
  if np.isnan(data[i]).any():
    index_to_remove.append(i)


index_to_remove = sorted(index_to_remove, reverse=True)
for i in index_to_remove:
  data.pop(i)
  labels.pop(i)

In [ ]:
# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [ ]:
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.4182721645889635


In [ ]:
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print("Precision:", precision)
print("Recall:", recall)

### Tune the SVM model using the parameter C

In [ ]:

# Defining the range of C values to consider
C_range = [0.1, 1, 10]

# Creating a dictionary of parameter values to search
param_grid = {'C': C_range}

# Defining the SVM model
svm = SVC(kernel='linear')

# Performing a grid search to find the best value of C
grid = GridSearchCV(svm, param_grid, cv=2, verbose=3)
grid.fit(X_train, y_train)

# Print the best value of C and the corresponding accuracy score
print("Best C:", grid.best_params_['C'])
print("Accuracy:", grid.best_score_)


Fitting 2 folds for each of 3 candidates, totalling 6 fits
[CV 1/2] END .............................C=0.1;, score=0.412 total time=10.1min
[CV 2/2] END .............................C=0.1;, score=0.410 total time= 9.8min
[CV 1/2] END ...............................C=1;, score=0.421 total time= 9.6min
[CV 2/2] END ...............................C=1;, score=0.419 total time= 9.7min
[CV 1/2] END ..............................C=10;, score=0.422 total time=11.7min
[CV 2/2] END ..............................C=10;, score=0.421 total time=11.7min
Best C: 10
Accuracy: 0.42146328705620817


### Read Glove embeddings

In [ ]:
word_to_embeddings = read_embeddings('GloVe_Embeddings (1).txt')

## Using GLoVe embeddings

In [ ]:
# convert lyrics to vectors based on the word embeddings
data = [] 
labels = [] 

for _, row in df.iterrows(): 
  label = row["Genre"] 
  lyrics = row["LyricsList"] 
  feature_vector = np.zeros((200,)) 
  song_length = 0 
  for line in lyrics: 
    song_length += len(line) 
    for word in line: 
      try:
        feature_vector += word_to_embeddings[word] 
      except:
        pass
  data.append(feature_vector/song_length) 
  labels.append(label)

<ipython-input-21-22ed93fe98d8>:17: RuntimeWarning: invalid value encountered in true_divide
  data.append(feature_vector/song_length)


In [ ]:
# check for nan values in data
# if present, remove that row in data and labels
index_to_remove = list()
c = 0
for i in range(len(data)):
  if np.isnan(data[i]).any():
    index_to_remove.append(i)


index_to_remove = sorted(index_to_remove, reverse=True)
for i in index_to_remove:
  data.pop(i)
  labels.pop(i)

In [ ]:
# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.25664719728009766
Precision: 0.06586778387172937
Recall: 0.25664719728009766


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Reading improved GloVe embeddings

In [13]:
# load word embedding files
def read_embeddings(training_file):
    '''Loads and parses embeddings trained in earlier.
    Parameters and return values are up to you.
    '''
    # you may find generating the following two dicts useful:
    # word to embedding : {'the':[0....], ...}
    # index to embedding : {1:[0....], ...} 
    # use your tokenizer's word_index to find the index of
    # a given word
    model = KeyedVectors.load_word2vec_format(training_file, binary=False)
    
    # Create a dictionary that maps words to their embeddings
    word_to_embeddings = {word: model[word] for word in model.key_to_index}

    # Create a dictionary that maps indexes to their embeddings
    # index_to_embeddings = {index: model.wv.vectors[index] for index in range(len(model.vocab))}
    
    return word_to_embeddings

In [17]:
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = "glove.6B.200d.txt"
tmp_file = get_tmpfile("test_word2vec.txt")
_ = glove2word2vec(glove_file, tmp_file)
glove_model = KeyedVectors.load_word2vec_format(tmp_file)
glove_word_to_embeddings = {word: glove_model[word] for word in glove_model.key_to_index}

<ipython-input-17-a08680d1240c>:7: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  _ = glove2word2vec(glove_file, tmp_file)


In [18]:
# convert lyrics to vectors based on the word embeddings
data = [] 
labels = [] 

for _, row in df.iterrows(): 
  label = row["Genre"] 
  lyrics = row["LyricsList"] 
  feature_vector = np.zeros((200,)) 
  song_length = 0 
  for line in lyrics: 
    song_length += len(line) 
    for word in line: 
      try:
        feature_vector += glove_word_to_embeddings[word] 
      except:
        pass
  data.append(feature_vector/song_length) 
  labels.append(label)

<ipython-input-18-ed6ab45c3a86>:17: RuntimeWarning: invalid value encountered in true_divide
  data.append(feature_vector/song_length)


In [19]:
# check for nan values in data
# if present, remove that row in data and labels
index_to_remove = list()
c = 0
for i in range(len(data)):
  if np.isnan(data[i]).any():
    index_to_remove.append(i)


index_to_remove = sorted(index_to_remove, reverse=True)
for i in index_to_remove:
  data.pop(i)
  labels.pop(i)

In [21]:
# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.3987010722692006
Precision: 0.3005364445028348
Recall: 0.3987010722692006


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
